In [1]:
import os
import sys

#  Set the working directory to the folder containing the top-level ultralytics package
os.chdir("/workspace")  # change to your workspace root where ultralytics folder exists
print("Current working directory:", os.getcwd())


Current working directory: /workspace


In [2]:
#  Add the top-level ultralytics folder to Python path
os.chdir("/workspace/ultralytics")
print("Current working directory:", os.getcwd())

Current working directory: /workspace/ultralytics


In [ ]:
from ultralytics import YOLO
import time
import torch

# === Parameters ===
dataset_yaml = "/workspace/datasets/KITTI/kitti.yml"
run_name = "yolov8n_base_patience20"
epochs = 250                # max epochs; early stopping may stop earlier
imgsz = 1280              # research-level resolution
batch_size = 32           # adjust according to VRAM
workers = 2               # safer for shared memory
device = "cuda:0" if torch.cuda.is_available() else "cpu"
cache_images = "disk"
amp = True                # mixed precision
save_interval = 40         # save checkpoint every 2 epochs
patience = 20             # early stopping patience

# === Initialize YOLOv8n ===
model = YOLO("yolov8n.pt")  # pre-trained weights


# === Start training ===
start_time = time.time()
print(f"Starting training for {epochs} epochs...\n")

results = model.train(
    data=dataset_yaml,
    epochs=epochs,
    imgsz=imgsz,
    batch=batch_size,
    workers=workers,
    device=device,
    cache=cache_images,
    name=run_name,
    save=True,
    amp=amp,
    patience=patience,
    save_period=save_interval
)

total_time = time.time() - start_time
print(f"\nTraining completed in {total_time/3600:.2f} hours")


Starting training for 250 epochs...

New https://pypi.org/project/ultralytics/8.3.202 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.200 🚀 Python-3.10.12 torch-2.4.0a0+f70bd71a48.nv24.06 CUDA:0 (NVIDIA A100-SXM4-80GB, 81051MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/workspace/datasets/KITTI/kitti.yml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=250, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1280, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mo

In [24]:
import sys
from pathlib import Path
import torch
import time
from ultralytics import YOLO  # Use YOLOv8's interface

weights = "runs/detect/yolov8n_base_patience20/weights/best.pt"
dataset_yaml = "/workspace/datasets/KITTI/kitti.yml"
results_dir = "/workspace/yolov8n_base_sdgpat_20val"

# ------------------- Device -------------------
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ------------------- Load model -------------------
model = YOLO(weights)  # Load the model using YOLOv8
params = sum(p.numel() for p in model.model.parameters())
params_mb = params * 4 / (1024**2)  # float32 -> 4 bytes
model_size_mb = Path(weights).stat().st_size / 1024**2  # file size in MB

print(f"Model size (file): {model_size_mb:.2f} MB")
print(f"Model parameters: {params_mb:.2f} MB")

# ------------------- FPS Measurement -------------------
dummy_input = torch.randn(1, 3, 1280, 1280).to(device) / 255.0  # Normalize the dummy input
# Warm-up
for _ in range(5):
    _ = model(dummy_input)  # Use the model directly for inference

n_runs = 50
start_time = time.time()
for _ in range(n_runs):
    _ = model(dummy_input)  # Use the model directly for inference
end_time = time.time()

fps = n_runs / (end_time - start_time)
print(f"Inference FPS (1280x1280): {fps:.2f}")

# ------------------- Run Validation -------------------
print("\nRunning YOLOv8 validation...")
start_val_time = time.time()
model.val(
    data=dataset_yaml,  # Validation dataset
    imgsz=1280,          # Image size
    batch=64,            # Corrected batch size argument
    device=device,       # Device
    project=results_dir, # Results directory
    name="eval_metrics", # Name for saved results
    save_json=True,      # Save results as JSON
    exist_ok=True,       # Overwrite if results directory exists
    verbose=True         # Show verbose output
)
end_val_time = time.time()

val_time = end_val_time - start_val_time
print(f"Validation time: {val_time:.2f} seconds")
print(f"Validation results saved to {results_dir}")


Model size (file): 5.99 MB
Model parameters: 11.49 MB

0: 1280x1280 (no detections), 5.3ms
Speed: 0.0ms preprocess, 5.3ms inference, 1.6ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.3ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.4ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)

0: 1280x1280 (no detections), 5.0ms
Speed: 0.0ms preprocess, 5.0ms inference, 1.2ms postprocess per image at shape (1, 3, 1280, 1280)



In [9]:
import shutil

# Specify the folder path and the destination zip file path
folder_path = 'runs/detect/yolov8n_base_patience20'
zip_file_path = 'runs/detect/yolov8n_base_patience20.zip'

# Create a zip file from the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/workspace/ultralytics/runs/detect/yolov8n_base_patience20.zip'

In [10]:
import shutil

# Specify the folder path and the destination zip file path
folder_path = '/workspace/vals/yolov8n_base_sdgpat_20val'
zip_file_path = '/workspace/vals/yolov8n_base_sdgpat_20val.zip'

# Create a zip file from the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/workspace/vals/yolov8n_base_sdgpat_20val.zip'